In [1]:
import os

import pandas as pd

In [2]:
# # объединение и сохранение всех результатов
# protostomia = pd.concat(
#     [
#         pd.read_csv("../results_summary/datasets_Cnidaria_cds_cassette.tsv", sep="\t"),
#         pd.read_csv("../results_summary/datasets_Ecdysozoa_cds_cassette.tsv", sep="\t"),
#         pd.read_csv("../results_summary/datasets_Spiralia_cds_cassette.tsv", sep="\t"),
#         pd.read_csv("../results_summary/psi_blast_Anthozoa_cds_cassette.tsv", sep="\t"),
#         pd.read_csv("../results_summary/psi_blast_Bivalvia_cds_cassette.tsv", sep="\t"),
#         pd.read_csv("../results_summary/psi_blast_Gastropoda_cds_cassette.tsv", sep="\t"),
#         pd.read_csv("../results_summary/psi_blast_Nematoda_cds_cassette.tsv", sep="\t"),
#         pd.read_csv("../results_summary/psi_blast_Platyhelminthes_cds_cassette.tsv", sep="\t"),
#     ], ignore_index=True
# )
#
# # ищем дубликаты
# # protostomia[protostomia["org_name"].duplicated(keep=False)]
#
# # удаляем дубликаты
# # -----------------
# # 64 - Actinia_tenebrosa__XP_031555922.1
# # 111 - Aplysia_californica__XP_005094816.1
# # 88 - Crassostrea_virginica__XP_022333190.1
# # 95 - Magallana_gigas__XP_065942772.1
# # -----------------
# indices_to_drop = [64, 111, 88, 95]
# protostomia = protostomia.drop(indices_to_drop)
#
# deuterostomia = pd.concat(
#     [
#         pd.read_csv("../results_summary/datasets_Actinopterygii_cds_cassette.tsv", sep="\t"),
#         pd.read_csv("../results_summary/datasets_Amphibia_cds_cassette.tsv", sep="\t"),
#         pd.read_csv("../results_summary/datasets_Sauropsida_cds_cassette.tsv", sep="\t"),
#         pd.read_csv("../results_summary/psi_blast_Caudata_cds_cassette.tsv", sep="\t"),
#         pd.read_csv("../results_summary/psi_blast_Gymnophiona_cds_cassette.tsv", sep="\t"),
#     ], ignore_index=True
# )
#
# # ищем дубликаты
# # deuterostomia[deuterostomia["org_name"].duplicated(keep=False)]
#
# # удаляем дубликаты
# # -----------------
# # 172 - Pleurodeles_waltl__KAJ1108515.1
# # 173 - Rhinatrema_bivittatum__XP_029448920.1
# # -----------------
# indices_to_drop = [172, 173]
# deuterostomia = deuterostomia.drop(indices_to_drop)
#
# # сохраняем итоговые таблицы
# protostomia.to_csv("../results_summary/protostomia.tsv", index=False, sep="\t")
# deuterostomia.to_csv("../results_summary/deuterostomia.tsv", index=False, sep="\t")

In [134]:
def create_table_for_phylum(df: pd.DataFrame, phylum: str, columns_to_select: list, column_to_sort: str,
                            save_dir: str = None) -> pd.DataFrame | None:
    df_subset = df[df.phylum == phylum].copy()
    df_selected = df_subset[columns_to_select]
    df_selected = df_selected[columns_to_select]

    df_selected["org_name"] = df_selected["org_name"].str.replace("_", " ")
    df_selected = df_selected.sort_values(by=column_to_sort, ascending=True)

    df_selected = df_selected.rename(columns={
        "org_name": "Название\nорганизма",
        "intron_length_to_stop_codon": "Кол-во\nнуклеотидов\nдо стоп-кодона\nв интроне",
        "first_exon_length": "Длина\n1-го экзона\nв кассете",
        "intron_length": "Длина\nкассетного\nинтрона",
        "second_exon_length": "Длина\n2-го экзона\nв кассете",
    })

    if save_dir is not None:
        df_selected.to_csv(f"{save_dir}/{phylum}.tsv", sep="\t", index=False, encoding="utf-8")
        return None
    else:
        return df_selected

### Первичноротые + книдарии

In [135]:
protostomia = pd.read_csv("../results_summary/protostomia.tsv", sep="\t")  # 181 вид, 3 филы

# добавляем phylum колонку в таблицу
phylum_list = []
for row in protostomia.iterrows():
    sub_phylum = row[1].sub_phylum
    match sub_phylum:
        case "Anthozoa":
            phylum = "Cnidaria"
        case "Nematoda" | "Scalidophora" | "Chelicerata" | "Crustacea" | "Polyneoptera" | "Paraneoptera" | "Endopterygota":
            phylum = "Ecdysozoa"
        case "Platyhelminthes" | "Bivalvia" | "Gastropoda":
            phylum = "Spiralia"
    phylum_list.append(phylum)
protostomia.insert(0, "phylum", phylum_list)

# добавляем длину второго экзона для красоты в таблице
protostomia["second_exon_length"] = 37

In [136]:
columns_to_select = ["org_name", "intron_length_to_stop_codon", "first_exon_length", "intron_length", "second_exon_length"]

for phylum in protostomia.phylum.unique():
    create_table_for_phylum(protostomia,
                            phylum=phylum,
                            columns_to_select=columns_to_select,
                            column_to_sort="intron_length_to_stop_codon",
                            save_dir="../results_print")

### Вторичноротые

In [137]:
deuterostomia = pd.read_csv("../results_summary/deuterostomia.tsv", sep="\t")  # 172 вида, 4 филы

# добавляем phylum колонку в таблицу
phylum_list = []
for row in deuterostomia.iterrows():
    sub_phylum = row[1].sub_phylum
    match sub_phylum:
        case "Otomorpha" | "Euteleosteomorpha" | "Osteoglossomorpha" | "Cladistia":
            phylum = "Actinopterygii"
        case "Gymnophiona" | "Caudata" | "Anura":
            phylum = "Amphibia"
        case "Lepidosauria":
            phylum = "Lepidosauria"
        case "Testudinata" | "Crocodylia" | "Aves":
            phylum = "Sauropsida"
    phylum_list.append(phylum)
deuterostomia.insert(0, "phylum", phylum_list)

# добавляем длину второго экзона для красоты в таблице
deuterostomia["second_exon_length"] = 37

In [138]:
columns_to_select = ["org_name", "intron_length_to_stop_codon", "first_exon_length", "intron_length", "second_exon_length"]

for phylum in deuterostomia.phylum.unique():
    create_table_for_phylum(deuterostomia,
                            phylum=phylum,
                            columns_to_select=columns_to_select,
                            column_to_sort="intron_length_to_stop_codon",
                            save_dir="../results_print")